In [1]:
from Bio import Entrez, SeqIO
import math
import pandas as pd
import time
import pysam
import json
import subprocess



genome = pysam.Fastafile('Mus_musculus.GRCm38.dna.primary_assembly.fa')

def subtract_substrings(string, substrings):
    for substring in substrings:
        string = string.replace(substring, "")
    return string


def getSeqfromChromosomeCoordinate(chrN, start, end,strand):
    seq = genome.fetch(chrN, start, end)
    if strand == '-':
        seq = seq[::-1]

    return seq


exonsinTrID = {}
tc = {}
strandInfoTrID={}
with open('Mus_musculus.GRCm39.109.gtf', 'r') as file:
#find exons and stop codons
    for line in file:
        if line.startswith('#'):
            continue

        fields = line.strip().split('\t')
        feature_type = fields[2]
        transcript_id = None
        chromosome = fields[0]
        start = int(fields[3])
        end = int(fields[4])
        strand = fields[6]
        attributes = fields[8].split(';')

        if feature_type == 'exon':
            for attr in attributes:
                if attr.strip().startswith('transcript_id'):
                    transcript_id = attr.split('"')[1].split('.')[0]
                    break

            if transcript_id is None:
                continue
            if not transcript_id in exonsinTrID:
                exonsinTrID[transcript_id] = {}
            exonsinTrID[transcript_id][start]=getSeqfromChromosomeCoordinate(chromosome, start, end, strand)
            strandInfoTrID[transcript_id] = strand
            
            
        if feature_type == 'stop_codon':
            for attr in attributes:
                if attr.strip().startswith('transcript_id'):
                    transcript_id = attr.split('"')[1].split('.')[0]
                    break

            if transcript_id is None:
                continue
            if not transcript_id in tc:
                tc[transcript_id] = []
            tc[transcript_id].append(start)

for tr in strandInfoTrID:
    exonStart = sorted(exonsinTrID[tr].keys())[0]

    exonsinTrID[tr] = [value for key, value in sorted(exonsinTrID[tr].items())]
    exonsinTrID[tr]='|i|'.join(exonsinTrID[tr])
    
    if tr in tc:
        for singleTC in tc[tr]:
            dist = singleTC - exonStart

            #given_string = "abcdefghi"
            #pos = 1
            exonsinTrID[tr] = exonsinTrID[tr][:dist] + "|t|" + exonsinTrID[tr][dist+3:]

    # Join the values into a single string
    if strandInfoTrID[tr]=='-':
        exonsinTrID[tr] = exonsinTrID[tr][::-1]
        continue
    elif strandInfoTrID[tr]=='+':
        continue
    print('neither - or +??'+str(tr))
    
    

In [2]:
import pandas as pd
from tqdm import tqdm

def getSeq(id):
    id = id.split('.')[0]
    if id in exonsinTrID:
        seq = exonsinTrID[id]
        return seq
    return False

emxi_8 = pd.read_csv("GSE220462_Upf2KO_mm/kallisto/Emxi_8/abundance.tsv", sep="\t")  # ctl
emxi_9 = pd.read_csv("GSE220462_Upf2KO_mm/kallisto/Emxi_9/abundance.tsv", sep="\t")  # ctl
emxi_4 = pd.read_csv("GSE220462_Upf2KO_mm/kallisto/Emxi_4/abundance.tsv", sep="\t")  # k0
emxi_5 = pd.read_csv("GSE220462_Upf2KO_mm/kallisto/Emxi_5/abundance.tsv", sep="\t")  # ko

seqCountko = {}
seqCountctl = {}
print('done reading')

filterD = pd.read_csv("intersect.bed", sep="\t", header=None)
fD = filterD[3].tolist()

# Progress bar for ctl processing
count = 0
with tqdm(total=len(emxi_8["target_id"]), desc="Processing ctl") as pbar_ctl:
    for target_id in emxi_8["target_id"]:
        valInEmxi9 = emxi_9[emxi_9["target_id"] == target_id]["tpm"].tolist()[0]
        valInEmxi8 = emxi_8[emxi_8["target_id"] == target_id]["tpm"].tolist()[0]
        avg = (valInEmxi8 + valInEmxi9) / 2
        seqCountctl[target_id.split('.')[0]] = avg
        count += 1
        pbar_ctl.update(1)
        pbar_ctl.set_description(f"Processing ctl: {count}/{len(emxi_8['target_id'])} ({100 * count / len(emxi_8['target_id']):.2f}%)")

print('done parsing1')

# Progress bar for ko processing
count = 0
with tqdm(total=len(emxi_5["target_id"]), desc="Processing ko") as pbar_ko:
    for target_id in emxi_4["target_id"]:
        valInEmxi4 = emxi_4[emxi_4["target_id"] == target_id]["tpm"].tolist()[0]
        valInEmxi5 = emxi_5[emxi_5["target_id"] == target_id]["tpm"].tolist()[0]
        avg = (valInEmxi4 + valInEmxi5) / 2
        seqCountko[target_id.split('.')[0]] = avg
        count += 1
        pbar_ko.update(1)
        pbar_ko.set_description(f"Processing ko: {count}/{len(emxi_5['target_id'])} ({100 * count / len(emxi_5['target_id']):.2f}%)")

print('done parsing2')

done reading


Processing ctl: 119414/119414 (100.00%): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 119414/119414 [28:58<00:00, 68.71it/s]


done parsing1


Processing ko: 119414/119414 (100.00%): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 119414/119414 [28:22<00:00, 70.15it/s]

done parsing2


In [3]:


import os
from tqdm import tqdm

res = {}
seqLen = len(seqCountctl)
count = 0

with tqdm(total=seqLen, desc="Processing") as pbar:
    for seq in seqCountko:
        numInKO = seqCountko[seq]
        numInCtl = 0

        if seq in seqCountctl:
            numInCtl = seqCountctl[seq] 

        if numInKO == 0  or ((seqCountctl[seq] <= 5 and seqCountko[seq] <= 5)):
            pbar.update(1)
            continue

        ret = getSeq(seq)
        if not ret:
            pbar.update(1)
            continue

        if numInCtl / numInKO > 1 and seq not in fD:
            nmd = '|false|'
        elif numInCtl / numInKO < 1 and seq in fD:
            nmd = '|true|'
        else:
            pbar.update(1)
            continue

        os.system("echo '" + ret + " |=| " + str(nmd) + "' >> rawDataSetIntronTCHighQualFineTune.txt &")
        count += 1
        pbar.set_description(f"Processing {count}/{seqLen} ({100 * count / seqLen:.2f}%)")
        pbar.update(1)
        
for i in exonsinTrID:
    os.system("echo '" + exonsinTrID[i]+"' >> rawDataSetIntronTCHighQualPretrain.txt &")

Processing 9585/119414 (8.03%): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 119414/119414 [01:37<00:00, 1222.89it/s]


In [4]:
!cat rawDataSetIntronTCHighQualFineTune.txt >> rawDataSetIntronTCHighQualPretrain.txt

In [5]:
'<i>'[::-1]

'>i<'